In [1]:
import numpy as np, pandas as pd, torch,os, glob, pickle
from sentence_transformers import SentenceTransformer, util
import tensorflow_hub as hub
import tensorflow_text
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
ST_BERT = text_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
COS_ = 1 # COS_=1: only sentence data, =2: do COS function 
model_name = "ST-DBMCv1"

2023-06-06 18:09:22.137676: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 18:09:22.288634: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-06 18:09:22.874711: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-06 18:09:22.874786: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
save_path = '/ssd8/chih/project/yadong/predict_baseline_version02_ACM_stage1/other_experiment/dataset/baseline-testdataset_pickle_save'
if COS_==0:
    save_folder = 'sample_500_seed_42_feature'
else:
    save_folder = 'sample_500_seed_42_feature_only_sentence'
save_model_type = model_name
os.makedirs(os.path.join(save_path, save_folder, save_model_type), exist_ok=True)

## ADD doctor free-text label

In [3]:
testdata_mark_periHD = pd.read_csv("../dataset/PeriHD_ExternalData_05-17.csv")
testdata_mark_intraHD = pd.read_csv("../dataset/IntraHD_ExternalData_05-17.csv")
testdata_mark_periHD['病人透析時間'] = pd.to_datetime(testdata_mark_periHD['病人透析時間'])
testdata_mark_intraHD['病人透析時間'] = pd.to_datetime(testdata_mark_intraHD['病人透析時間'])
print(testdata_mark_periHD.shape, testdata_mark_intraHD.shape)

(497, 4) (497, 4)


In [6]:
#pickle loading
USD_data_test_pkl = glob.glob("./dataset/baseline-testdataset_pickle_save/sample_500_seed_42/*.pkl")
break_count = 0
for next_pkl in (USD_data_test_pkl):
    save_list = os.listdir(os.path.join(save_path,save_folder))
    with open(next_pkl, 'rb') as f:
        singel_patient_df = pickle.load(f)
        single_patient_periHD = singel_patient_df['PeriHD']
        single_patient_intraHD = singel_patient_df['IntraHD']
        try:
            intraHD_same_time = testdata_mark_intraHD[testdata_mark_intraHD['病人透析時間']==list(single_patient_intraHD['洗腎紀錄時間'])[0]]
            intraHD_answer = intraHD_same_time.label.values[0]
            peirHD_same_time = testdata_mark_periHD[testdata_mark_periHD['病人透析時間']==single_patient_periHD['洗腎紀錄時間']]
            periHD_answer = peirHD_same_time.label.values[0]
        
        except:
            if single_patient_periHD['ID'] == "06821J282":
                periHD_answer = 0
                intraHD_answer = 0
            if single_patient_periHD['ID'] == "7845777":
                periHD_answer = 0
                intraHD_answer = 0
        # print("I_ANS: {} P_ANS: {}".format(intraHD_answer, periHD_answer))
        # 轉換IntraHD和PeriHD的斷句特徵
        # IntraHD
        save_name = "baseline_01_sample_500_feature_ID-{}-DateTime-{}.pkl".format(singel_patient_df['ID'], str(singel_patient_df['DateTime']).replace(':','-').replace(' ','-'))
        if save_name not in save_list:
            # break
            IntraHD_data = single_patient_intraHD['處置其他+症狀處置（描述）'].astype(str).to_list()

            IntraHD_feature = ST_BERT.encode(IntraHD_data)
            # PeriHD處置其他結束
            if COS_==0:
                PeriHD_data = str(single_patient_periHD['處置其他結束'])
                PeriHD_feature = ST_BERT.encode(PeriHD_data)
            elif COS_==1:
                text = str(single_patient_periHD['處置其他結束']).replace('\t','').replace('\n','').replace('。','，').replace(',','，')
                text_sp = np.array([i for i in text.split('，') if len(i)>1])
                PeriHD_data = []
                for i in range(len(text_sp)-1):
                    PeriHD_data.append(text_sp[i])
                if len(text_sp)==1:
                    PeriHD_data.append(text_sp[0])
                PeriHD_feature = ST_BERT.encode(PeriHD_data)
            with open(os.path.join(save_path, save_folder, save_model_type, "baseline_01_sample_500_feature_ID-{}-DateTime-{}.pkl".format(singel_patient_df['ID'], str(singel_patient_df['DateTime']).replace(':','-').replace(' ','-'))), "wb") as f:
                new_pickle_feature = {'ID':singel_patient_df['ID'], 'DateTime': singel_patient_df['DateTime'], 
                                    'PeriHD_data_sbp':single_patient_periHD['NEW開始血壓SBP'],
                                    'PeriHD_data_dbp':single_patient_periHD['NEW開始血壓DBP'],
                                    'PeriHD_Feature': PeriHD_feature, 
                                    'PeriHD_context': PeriHD_data,
                                    'PeriHD_answer': periHD_answer, 
                                    'IntraHD_data_sbp':single_patient_intraHD['sbp'],
                                    'IntraHD_data_dbp':single_patient_intraHD['dbp'],                        
                                    'IntraHD_Feature': IntraHD_feature,
                                    'IntraHD_context': IntraHD_data,
                                    'IntraHD_answer': intraHD_answer,
                                    'IDH_answer':1 if periHD_answer+intraHD_answer!=0 else 0}
                # break
                pickle.dump(new_pickle_feature, f)
            # del new_pickle_feature, IntraHD_feature, PeriHD_feature
            # break